# PIPELINE 그래프 예제

일반적인 케이스로 구성되어 있는 파이프라인을 작성합니다.<br>
각 컴포넌트는 파이프라인의 워크플로우 파악을 위하여 입력 값에 컴포넌트 작업을 추가하여 출력 값을 반환합니다.<br>
<span style = 'font-weight:bold'>본 예제는 아래 그래프로 구성된 파이프라인 구현을 목표로 합니다.</span>

<img src='node_image/pipeline_example.png' width='300px'></img>

예제 구현 전 기본적인 컴포넌트 연결 구성을 먼저 확인하여 파이프라인을 작성한 후 확장하여 구현합니다.

In [19]:
import kfp
from kfp import components, dsl
from kfp.components import func_to_container_op

## 컴포넌트 2개

먼저 두 개의 컴포넌트로 구성한 파이프라인을 작성합니다. 파이프라인 작성에 필요한 노드 A, B 컴포넌트를 선언합니다.<br>
노드A는 그래프의 시작이므로 출력 값만 정의합니다.<br>
노드B는 입력과 출력을 모두 정의합니다.

In [20]:
@func_to_container_op
def node_A() -> str:
    task_A = 'A'
    print(task_A)
    return task_A

@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

## 파이프라인 작성 (컴포넌트 2개)

두 노드를 연결하는 파이프라인을 작성합니다. 먼저 노드A 작업을 수행하는 인스턴스를 정의합니다.<br>
다음으로 노드A의 출력값을 입력으로 받는 노드B를 정의합니다.<br>
두 노드는 정의되어 있는 입력과 출력을 작성자가 연결해줌으로써 그래프를 완성합니다.

In [21]:
@dsl.pipeline(name='pipeline example')
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.output)

In [22]:
kfp.compiler.Compiler().compile(connect_example_pipeline, 'node2_example.yaml')

### 파이프라인 결과

작성한 파이프라인을 kubeflow에 업로드하여 결과 화면을 확인합니다.
노드A의 출력을 확인할 수 있습니다.

![node_image/nodeA.png](node_image/nodeA.png)

노드B는 노드A의 출력을 입력으로 받은 후 출력을 통해 파이프라인의 워크플로우를 확인할 수 있습니다.

![node_image/nodeB.png](node_image/nodeB.png)

## 컴포넌트 3개

두개의 컴포넌트를 파이프라인으로 구성함으로써 파이프라인의 단일 연결 방법은 확인하였습니다.<br>
컴포넌트는 출력값을 확실히 정의해주어야 하기 때문에 위 예제에서는 string으로 노드의 출력 값을 확인합니다.<br>
하지만 2개 이상의 출력 값은 list, tuple 형태의 데이터를 활용해야 합니다.<br>
<br>
노드A는 tuple 형태의 출력값을 가집니다. 이를 사용하기 위하여 NamedTuple 패키지를 활용합니다.

In [23]:
from typing import NamedTuple

kubeflow pipeline에서는 입력값과 출력값을 확실히 정의해야되기 때문에 NamedTuple을 사용하여 다음과 같이 노드A를 새롭게 정의합니다.

In [24]:
@func_to_container_op
def node_A() -> NamedTuple('Outputs', [('first', str), ('second', str)]):
    task_A = 'A'
    print(task_A)
    return (task_A, task_A)

노드B는 기존과 같은 작업을 진행하기 때문에 재사용합니다. 그리고 노드B와 같은 작업을 수행하는 노드C를 새롭게 정의합니다.

In [25]:
@func_to_container_op
def node_C(C: str) -> str:
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

## 파이프라인 작성 (컴포넌트 3개)

이제 3개의 컴포넌트를 연결하는 파이프라인을 새롭게 작성합니다. 단일 출력일 시 output를 사용하지만 출력이 2개 이상일 때는 outputs를 사용한 후 원하는 출력의 key값을 작성합니다.

In [26]:
@dsl.pipeline(name='pipeline example')
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.outputs['first'])
    node_C_task = node_C(node_A_task.outputs['second'])

In [27]:
kfp.compiler.Compiler().compile(connect_example_pipeline, 'node3_example.yaml')

### 파이프라인 결과

A -> B, A -> C의 워크플로우를 확인할 수 있습니다.

![node_image/nodeC.png](node_image/nodeC.png)

# 예제 그래프 작성

컴포넌트 작성 예제를 통해 다음 그래프로 구성되어 있는 파이프라인을 작성합니다.<br>
현재 구성되어 있는 노드A,B,C에서 노드D,E,F를 추가합니다.<br>
노드D,E는 노드B의 출력을 입력으로 연결이 구성되어 있습니다.<br> 
먼저 노드B를 노드A의 출력과 같이 수정합니다. 단, 노드B는 입력값이 존재하는 것을 유의합니다.

In [28]:
@func_to_container_op
def node_B(B: str) -> NamedTuple('Outputs', [('first', str), ('second', str)]):
    task_B = f'{B} -> B'
    print(task_B)
    return (task_B, task_B)

노드 B의 출력을 입력으로 받는 노드D,E도 정의합니다.

In [29]:
@func_to_container_op
def node_D(D: str) -> str:
    task_D = f'{D} -> D'
    print(task_D)
    return task_D

@func_to_container_op
def node_E(E: str) -> str:
    task_E = f'{E} -> E'
    print(task_E)
    return task_E

마지막으로 노드C의 출력을 입력으로 받는 노드F도 정의합니다.

In [30]:
@func_to_container_op
def node_F(F: str) -> str:
    task_F = f'{F} -> F'
    print(task_F)
    return task_F

## 파이프라인 작성 (예제 그래프)

예제 그래프를 구성하는 모든 노드(컴포넌트)들을 정의하였습니다.<br>
이제 각각의 입력과 출력을 연결하여 파이프라인을 완성해야 합니다. 

In [31]:
@dsl.pipeline(name='pipeline example')
def connect_example_pipeline():
    #시작 노드
    node_A_task = node_A()
    
    #왼쪽 구성 트리
    node_B_task = node_B(node_A_task.outputs['first'])

    node_D_task = node_D(node_B_task.outputs['first'])
    node_E_task = node_E(node_B_task.outputs['second'])

    #오른쪽 구성 트리
    node_C_task = node_C(node_A_task.outputs['second'])

    node_F_task = node_F(node_C_task.output)

In [32]:
kfp.compiler.Compiler().compile(connect_example_pipeline, 'DAG_node_example.yaml')

## 파이프라인 결과 (예제 그래프)

작성한 파이프라인을 kubeflow pipeline에 업로드 시 다음과 같이 예제와 같은 모양의 그래프를 확인할 수 있습니다.

<img src = 'node_image/kubeflow_pipeline_result.png'></img>

구성한 파이프라인의 워크플로우 확인을 위하여 NodeE와 NodeF를 확인합니다.

<img src = 'node_image/nodeE.png'></img>

<img src = 'node_image/nodeF.png'></img>

트리의 마지막 노드를 통하여 워크플로우를 확인할 수 있습니다.